<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220331_sm_SANUP_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 3.8 MB 70.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.1 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 212 kB 62.9 MB/s 
     |████████████████████████████████| 134 kB 70.5 MB/s 
     |████████████████████████████████| 127 kB 89.3 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 86.8 MB/s 
     |████████████████████████████████| 144 kB 88.0 MB/s 
     |████████████████████████████████| 6.5 MB 83.5 MB/s 
     |████████████████████████████████| 895 kB 80.3 MB/s 
     |████████████████████████████████| 596 kB 81.8 MB/s 
     |████████████████████████████████| 1.2 MB 78.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [5]:
from accelerate import Accelerator

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [7]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [8]:
_, df = train_test_split(df, test_size=0.1, shuffle=True)

In [9]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
732812,id_0732813,G,47,474,사업장에서,일반소비자대상으로,가방 마대자루판매
767754,id_0767755,I,56,561,음식업,접객시설갖추고,한식 일반음식점업
785019,id_0785020,C,23,239,"규석, 석회석, 조경석 가공",제조업,"규석, 석회석, 조경석"
576894,id_0576895,S,96,961,사업장에서,고객을대상으로,피부미용서비스
508076,id_0508077,G,47,476,서점,일반소비자대상으로,책
...,...,...,...,...,...,...,...
909704,id_0909705,L,68,682,일반인ㄴ대상,부동산중개소에서,"매매,전세계약"
338566,id_0338567,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스
923660,id_0923661,I,56,562,다방,커피.음료,비알콜음료점업
110033,id_0110034,C,10,102,덕장에서,명태를건조하여,명태코다리판매


In [10]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [11]:
df = df.fillna(0)

In [12]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
732812,id_0732813,G,47,474,사업장에서,일반소비자대상으로,가방 마대자루판매,사업장에서 일반소비자대상으로 가방 마대자루판매,G 47 474
767754,id_0767755,I,56,561,음식업,접객시설갖추고,한식 일반음식점업,음식업 접객시설갖추고 한식 일반음식점업,I 56 561
785019,id_0785020,C,23,239,"규석, 석회석, 조경석 가공",제조업,"규석, 석회석, 조경석","규석, 석회석, 조경석 가공 제조업 규석, 석회석, 조경석",C 23 239
576894,id_0576895,S,96,961,사업장에서,고객을대상으로,피부미용서비스,사업장에서 고객을대상으로 피부미용서비스,S 96 961
508076,id_0508077,G,47,476,서점,일반소비자대상으로,책,서점 일반소비자대상으로 책,G 47 476
...,...,...,...,...,...,...,...,...,...
909704,id_0909705,L,68,682,일반인ㄴ대상,부동산중개소에서,"매매,전세계약","일반인ㄴ대상 부동산중개소에서 매매,전세계약",L 68 682
338566,id_0338567,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스,개인택시로 일반인을 대상으로 승객운송서비스,H 49 492
923660,id_0923661,I,56,562,다방,커피.음료,비알콜음료점업,다방 커피.음료 비알콜음료점업,I 56 562
110033,id_0110034,C,10,102,덕장에서,명태를건조하여,명태코다리판매,덕장에서 명태를건조하여 명태코다리판매,C 10 102


In [13]:
train = df[['text', 'target']]
train

,text,target
732812,사업장에서 일반소비자대상으로 가방 마대자루판매,G 47 474
767754,음식업 접객시설갖추고 한식 일반음식점업,I 56 561
785019,"규석, 석회석, 조경석 가공 제조업 규석, 석회석, 조경석",C 23 239
576894,사업장에서 고객을대상으로 피부미용서비스,S 96 961
508076,서점 일반소비자대상으로 책,G 47 476
...,...,...
909704,"일반인ㄴ대상 부동산중개소에서 매매,전세계약",L 68 682
338566,개인택시로 일반인을 대상으로 승객운송서비스,H 49 492
923660,다방 커피.음료 비알콜음료점업,I 56 562
110033,덕장에서 명태를건조하여 명태코다리판매,C 10 102


In [14]:
train, valid = train_test_split(train, test_size=0.1, shuffle=True)

In [24]:
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [28]:
valid

,text,target
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475
1,교회학습관 기독교계통 종교서비스,S 94 949
2,사업장에서 소매업체에 도매 윤활유,G 46 467
3,제조업 산업용 자동화설비및 제어장치,C 27 272
4,부동산에서 계약에의거 부동산중개,L 68 682
...,...,...
9995,차량계량 0 0,H 52 529
9996,편의점에서 체인계약을 체결하여 음식료품 담배,G 47 471
9997,고객의시설에서 고객방문하여 학습지 교육서비스,P 85 855
9998,"사업장에서 소매업자에게 들기름,참기름 도매",G 46 463


# 토크나이저, 함수

In [15]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [16]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [17]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
325280,"지하상가에서 학생,일반인의 옷수선서비스",0
38548,음식점에서 접객시설을 갖추고 곰탕 국수판매,1
38514,음식점에서 접객시설을갖추고 천둥오리,1
728747,"소매업자에게 약초도매 한약재등을 이용하여 구기자,맥문동 판매",2
270360,사업장 조리하여 반찬제조,3
...,...,...
763000,회계업무를 의뢰받아 회계사사무소에서 회계업무,7
394702,영업장에서 접객시설을 갖추고 보리밥 돌솥밥 판매,1
407271,민박 0 0,59
804580,영업장에서 산업사용자에게 건설용석제품및 비석 도매,56


In [18]:
my_dict['0.0 0.0 0.0'] = len(my_dict)

In [19]:
train[train['text'].isnull()]

,text,target


In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [31]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220331_valid/model' + str(fold), save_function=accelerator.save)

In [32]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.2078900709219858
epoch 1 train acc 0.20833333333333334
epoch 1 train acc 0.2063386524822695
epoch 1 train acc 0.2116578014184397
epoch 1 train acc 0.2106604609929078
epoch 1 train acc 0.21431737588652483
epoch 1 train acc 0.21043882978723405
epoch 1 train acc 0.21143617021276595
epoch 1 test acc 0.304137323943662
epoch 1 test acc 0.31205985915492956
epoch 1 test acc 0.2953345070422535
epoch 1 test acc 0.3045774647887324
epoch 1 test acc 0.31161971830985913
epoch 1 test acc 0.3248239436619718
epoch 1 test acc 0.2935739436619718
epoch 1 test acc 0.29797535211267606
epoch 2 train acc 0.47285017730496454
epoch 2 train acc 0.47384751773049644
epoch 2 train acc 0.4763962765957447
epoch 2 train acc 0.48681294326241137
epoch 2 train acc 0.4811613475177305
epoch 2 train acc 0.48503989361702127
epoch 2 train acc 0.47816932624113473
epoch 2 train acc 0.48570478723404253
epoch 2 test acc 0.6144366197183099
epoch 2 test acc 0.6047535211267606
epoch 2 test acc 0.6223591549295775


# 추론

In [37]:
test = valid.fillna(0)

In [38]:
test['target'] = 'G 47 475'
test

,text,target
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475
1,교회학습관 기독교계통 종교서비스,G 47 475
2,사업장에서 소매업체에 도매 윤활유,G 47 475
3,제조업 산업용 자동화설비및 제어장치,G 47 475
4,부동산에서 계약에의거 부동산중개,G 47 475
...,...,...
9995,차량계량 0 0,G 47 475
9996,편의점에서 체인계약을 체결하여 음식료품 담배,G 47 475
9997,고객의시설에서 고객방문하여 학습지 교육서비스,G 47 475
9998,"사업장에서 소매업자에게 들기름,참기름 도매",G 47 475


In [34]:
# test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
# test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
# test = test[['text', 'target']]
# test

KeyError: ignored

In [40]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 216
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220331_valid/model' + str(fold), num_labels=216)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220331_valid/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220331_valid/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

In [ ]:
print(my_variable)

In [ ]:
submission

In [41]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_label4.csv')

In [44]:
test = valid

In [49]:
valid

,text,target,label0,label1,label2,label3,label4
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
1,교회학습관 기독교계통 종교서비스,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,사업장에서 소매업체에 도매 윤활유,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467
3,제조업 산업용 자동화설비및 제어장치,C 27 272,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292
4,부동산에서 계약에의거 부동산중개,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682
...,...,...,...,...,...,...,...
9995,차량계량 0 0,H 52 529,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493
9996,편의점에서 체인계약을 체결하여 음식료품 담배,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471
9997,고객의시설에서 고객방문하여 학습지 교육서비스,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855
9998,"사업장에서 소매업자에게 들기름,참기름 도매",G 46 463,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463


In [45]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [46]:
df.head(3)

,text,target,label0,label1,label2,label3,label4
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
1,교회학습관 기독교계통 종교서비스,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,사업장에서 소매업체에 도매 윤활유,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467


In [47]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

0.8947

In [48]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

,text,target,label0,label1,label2,label3,label4
7,"사무실에서 분식점 가맹점을 모집, 관리",M 71 715,F 41 411,F 41 411,L 68 681,F 41 411,F 41 411
11,크린장비 업체의 요구의따라 냉난방기제조,C 29 291,C 22 222,C 25 259,C 25 259,C 29 291,C 25 259
16,사업장에서 사용자주문에 스키고글 제조,C 27 271,C 25 259,C 22 222,C 22 222,C 22 222,C 22 222
22,렌트카 임대업 자동차렌트,N 76 761,K 64 641,H 52 529,H 52 529,H 52 529,H 52 529
29,시멘트.콘크리트 거래처의의뢰를받아 도로건설및토목건축,F 42 421,F 42 421,F 41 411,F 41 411,F 42 421,F 41 411
...,...,...,...,...,...,...,...
9990,조세관련 고객의 의뢰를 받아 기장대리 세무조정,M 71 712,M 71 711,M 71 711,M 71 711,M 71 712,M 71 711
9994,사업장에서 일반인 대상으로 천연염색 체험,R 91 912,R 91 911,S 96 961,R 91 911,R 91 911,R 91 911
9995,차량계량 0 0,H 52 529,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493
9998,"사업장에서 소매업자에게 들기름,참기름 도매",G 46 463,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463


In [50]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_valid/pred_prob4.csv')

In [51]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,206,207,208,209,210,211,212,213,214,215
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,0.000138,0.000285,0.002578,...,0.000024,0.000013,0.000011,0.000019,0.000023,0.000007,0.000011,0.000011,0.000016,0.000011
1,교회학습관 기독교계통 종교서비스,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,0.000164,0.001213,0.000162,...,0.000010,0.000004,0.000006,0.000007,0.000006,0.000004,0.000006,0.000007,0.000006,0.000008
2,사업장에서 소매업체에 도매 윤활유,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,0.001805,0.000438,0.008939,...,0.000071,0.000024,0.000034,0.000020,0.000062,0.000016,0.000041,0.000021,0.000043,0.000030


In [52]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,207,208,209,210,211,212,213,214,215,soft
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,0.000138,0.000285,0.002578,...,0.000013,0.000011,0.000019,0.000023,0.000007,0.000011,0.000011,0.000016,0.000011,G 47 478
1,교회학습관 기독교계통 종교서비스,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,0.000164,0.001213,0.000162,...,0.000004,0.000006,0.000007,0.000006,0.000004,0.000006,0.000007,0.000006,0.000008,S 94 949
2,사업장에서 소매업체에 도매 윤활유,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,0.001805,0.000438,0.008939,...,0.000024,0.000034,0.000020,0.000062,0.000016,0.000041,0.000021,0.000043,0.000030,G 46 467


In [53]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

,label0,label1,label2,label3,label4,soft
0,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
1,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467
3,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292
4,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682
...,...,...,...,...,...,...
9995,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493,H 49 493
9996,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471
9997,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855
9998,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463,G 46 467


In [54]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [55]:
soft[soft['soft'] != soft['hard']]

,label0,label1,label2,label3,label4,soft,temp,hard
29,F 42 421,F 41 411,F 41 411,F 42 421,F 41 411,F 42 421,"{'F 42 421': 2, 'F 41 411': 3}",F 41 411
69,C 33 339,C 22 222,C 22 222,C 33 339,C 33 339,C 22 222,"{'C 33 339': 3, 'C 22 222': 2}",C 33 339
88,F 41 411,L 68 681,N 75 751,L 68 681,F 41 411,F 41 411,"{'F 41 411': 2, 'L 68 681': 2, 'N 75 751': 1}",0
98,G 46 467,G 46 466,G 46 466,G 46 467,G 46 466,G 46 467,"{'G 46 467': 2, 'G 46 466': 3}",G 46 466
139,G 46 467,F 41 412,F 42 422,F 41 412,F 42 423,F 42 422,"{'G 46 467': 1, 'F 41 412': 2, 'F 42 422': 1, ...",0
...,...,...,...,...,...,...,...,...
9708,L 68 681,L 68 681,H 52 529,N 75 751,H 52 529,H 52 529,"{'L 68 681': 2, 'H 52 529': 2, 'N 75 751': 1}",0
9769,F 42 421,F 42 422,F 42 422,F 42 422,F 42 421,F 42 421,"{'F 42 421': 2, 'F 42 422': 3}",F 42 422
9774,C 25 259,C 22 222,C 22 222,C 22 222,C 25 259,C 25 259,"{'C 25 259': 2, 'C 22 222': 3}",C 22 222
9819,G 47 478,C 14 141,C 14 141,G 47 475,G 47 478,C 14 141,"{'G 47 478': 2, 'C 14 141': 2, 'G 47 475': 1}",0


In [57]:
valid['target']

,text,target,label0,label1,label2,label3,label4
0,일반소비자를대상으로 매장에서 LED조명 전구,G 47 475,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
1,교회학습관 기독교계통 종교서비스,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,사업장에서 소매업체에 도매 윤활유,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467
3,제조업 산업용 자동화설비및 제어장치,C 27 272,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292
4,부동산에서 계약에의거 부동산중개,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682
...,...,...,...,...,...,...,...
9995,차량계량 0 0,H 52 529,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493
9996,편의점에서 체인계약을 체결하여 음식료품 담배,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471
9997,고객의시설에서 고객방문하여 학습지 교육서비스,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855
9998,"사업장에서 소매업자에게 들기름,참기름 도매",G 46 463,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463


In [56]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

,label0,label1,label2,label3,label4,soft,hard
0,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
1,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467
3,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292
4,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682
...,...,...,...,...,...,...,...
9995,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493,H 49 493,H 49 493
9996,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471
9997,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855
9998,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463,G 46 467,G 46 463


In [58]:
df['real'] = valid['target']
df

,label0,label1,label2,label3,label4,soft,hard,real
0,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 475
1,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
2,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467,G 46 467
3,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 29 292,C 27 272
4,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682,L 68 682
...,...,...,...,...,...,...,...,...
9995,H 49 493,G 45 452,H 52 529,H 49 493,H 49 493,H 49 493,H 49 493,H 52 529
9996,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471,G 47 471
9997,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855,P 85 855
9998,G 46 463,G 46 467,G 46 467,G 46 463,G 46 463,G 46 467,G 46 463,G 46 463


In [59]:
len(df[df['soft'] == df['real']])/len(df)

0.7789

In [60]:
len(df[df['hard'] == df['real']])/len(df)

0.7782

In [70]:
df[(df['soft'] != df['real']) & (df['soft'] != df['hard'])][:50]

,label0,label1,label2,label3,label4,soft,hard,real
69,C 33 339,C 22 222,C 22 222,C 33 339,C 33 339,C 22 222,C 33 339,C 20 202
88,F 41 411,L 68 681,N 75 751,L 68 681,F 41 411,F 41 411,0,N 75 759
98,G 46 467,G 46 466,G 46 466,G 46 467,G 46 466,G 46 467,G 46 466,G 46 466
139,G 46 467,F 41 412,F 42 422,F 41 412,F 42 423,F 42 422,0,F 42 423
154,C 34 340,C 33 339,J 58 582,C 29 291,C 29 291,C 18 181,0,N 76 763
196,L 68 681,H 52 529,H 52 529,S 96 969,S 96 969,S 96 969,0,M 73 733
255,G 46 467,G 46 466,G 46 466,G 46 467,G 46 466,G 46 467,G 46 466,G 46 466
300,G 46 467,C 33 339,H 52 521,G 46 463,G 46 467,G 46 467,0,G 47 476
437,G 46 467,G 46 465,G 46 465,G 46 467,G 46 466,G 46 467,0,G 46 466
574,C 25 259,C 22 222,C 18 181,C 25 259,C 22 222,C 25 259,0,C 30 303


In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = pred['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = pred['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = pred['label'].map(split_label)